# Project 1: Food Vision Big

## Check GPU for Mixed Precision Training Capability

> The author of this notebook is running this notebook in Jupyter Lab on a home PC utilizing an NVIDIA RTX3090 GPU. For Mixed Precision Training to work, the GPU you're running must have a 'Compute Capability' of 7.0 or higher. That information can be found here for NVIDIA: https://developer.nvidia.com/cuda-gpus#compute

In [1]:
# Check which GPU you're using
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-27f8c828-87c9-7878-78e0-863228c3c5df)


## Get helper functions

This `helper_functions.py` is from other works. We can download it from here: https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

In [2]:
# Download helper_functions.py if it does not exist
import os
if not os.path.exists("helper_functions.py"):
  !wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
else:
  print("[INFO] 'helper_functions.py' exists, skipping download.")

--2023-01-18 02:57:32--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-01-18 02:57:32 (60.3 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
from helper_functions import create_tensorboard_callback, plot_loss_curves, compare_historys

## Use TensorFlow Datasets to Download Data

[Guide and datasets list here.](https://www.tensorflow.org/datasets)

In [4]:
# Get TensorFlow Dataset
import tensorflow_datasets as tfds

In [5]:
# turn off check for Google authentication for the public bucket these datasets reside in
tfds.core.utils.gcs_utils._is_gcs_disabled = True
os.environ['NO_GCE_CHECK'] = 'true'

# List all available datasets
datasets_list = tfds.list_builders() # get all available in TFDS

# Check if food101 is in the list
# print("food101" in datasets_list)

In [ ]:
# # Load in the Food101 data (this can take a few minutes, 5GB)
(train_data, test_data), ds_info = tfds.load(name="food101",
                                            split=["train", "validation"], # each data set is potentially different, some have train, valid, test
                                            shuffle_files=True,
                                            as_supervised=True, # data gets returns in tuple format (data, label)
                                            with_info=True) # this provides the data back to the ds_info variable

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

In [ ]:
# Features of Food101 from TFDS
ds_info.features

In [ ]:
# Get class names
class_names = ds_info.features["label"].names
class_names[:10]

## Exploring Food101 data from TFDS

We need to find:
* Class names
* Shape of image tensors
* Datatype of the data
* What do labels look like (one-hot vs label encoded)
* Do the labels match up with the class_names?

In [ ]:
# Take one sample of the train data
train_one_sample = train_data.take(1) # samples are in format (image_tensor, label)

In [ ]:
# What does one sample of our training data look like?
train_one_sample

In [ ]:
# Output info about the training sample
for image, label in train_one_sample:
    print(f"""
    Image Shape: {image.shape},
    Image datatype: {image.dtype},
    Target class from Food101 (tensor form): {label}
    Class name (str form): {class_names[label.numpy()]}
    """)

In [ ]:
# What does our image tensor from TFDS Food101 look like?
image

In [ ]:
# What are the min and max values of our image tensor? RGB 0-255, need to normalize
import tensorflow as tf
tf.reduce_min(image), tf.reduce_max(image)

### Plot an image from TensorFlow Datasets

In [ ]:
# Plot an image tensor
import matplotlib.pyplot as plt
plt.imshow(image)
plt.title(class_names[label.numpy()]) # add title to image to verify label is matched to training data correctly
plt.axis(False)

## Create preprocessing functions for our data

Neural Networks perform best when data is prepared in a certain way (e.g., batched, normalized, etc.)

What we know about our data:
* In `uint8` datatype
* Comprised of various sized tensors
* Not scaled (pixels are RGB values)

What models like:
* Data in `float32` or mixed precision `float16` and `float32`
* For batches, TensorFlow likes all of the tensors within a batch to be of the same size (typically 32)
* Scaled (values between 0 & 1), also called normalization

EfficientNetBX pretrained model will be used from `tf.keras.applications`, these models have a layer for scaling built in.

This means our functions need to:
1. Reshape our images to all the same size
2. Convert the dtype of our image tensors from `uint8` to `float32`

In [ ]:
# Make function for preprocessing images
def preprocess_img(image, label, img_shape=224):
    """
    Converts image datatype form 'uint8' -> 'float32' and reshapes
    image to [img_shape, img_shape, color_channels]
    """
    image = tf.image.resize(image, [img_shape, img_shape]) # reshape target image
    # image = image / 255. # scales image values, however, this is not required for EfficientNetBX models from tf.keras.applications as a normalization/scaling layer exists
    return tf.cast(image, tf.float32), label # return (float32_image, label) tuple

In [ ]:
# preprocess a single sample image and check outputs
preprocessed_img = preprocess_img(image, label)[0]
print(f"Image before preprocessing:\n {image[:2]}..., \nShape: {image.shape}, \nDatatype: {image.dtype}")
print(f"Image after preprocessing:\n {preprocessed_img[:2]}..., \nShape: {preprocessed_img.shape}, \nDatatype: {preprocessed_img.dtype}")

## Batch & Prepare datasets

Create a performant data input pipeline.

As a resource, the [tf.data API](https://www.tensorflow.org/guide/data) as well as [TesorFlow pipeline optimizations](https://www.tensorflow.org/guide/data_performance).

In [ ]:
# Map preprocessing function to training data (and parallelize)
train_data = train_data.map(map_func=preprocess_img, num_parallel_calls=tf.data.AUTOTUNE)

# Shuffle train_data and turn it into batches and prefect it (load it faster). buffer_size limits shuffling to limit RAM overload
train_data = train_data.shuffle(buffer_size=1000).batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE)

# Map preprocessing function to test data
test_data = test_data.map(preprocess_img, num_parallel_calls=tf.data.AUTOTUNE).batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
train_data, test_data

> The above cells maps this preprocessing function (`preprocess_img`) across our training dataset, then shuffle a number of elements, then we'll batch a collection of 32. Finally we'll prepare batches via 'prefetch' while the model is training on the previous batch.

## Create modeling callback

Callbacks to create:
* Tensorboard callback to log training results for visualization
* ModelCheckpoint callback to save our model's progress after feature extraction

In [ ]:
# Create TensorBoard callback (import from helper_functions.py)
from helper_functions import create_tensorboard_callback

# Create ModelCheckpoint callback to save a model's progress during training
checkpoint_path = "model_checkpoints/cp.ckpt"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                      monitor="val_acc",
                                                      save_best_only=True,
                                                     save_weights_only=True,
                                                     verbose=0) # don't print to stdout

## Setup mixed precision training

[Mixed Precision Documentation](https://www.tensorflow.org/guide/mixed_precision)

Mixed precision utilized a mix of float16 and float32 to improve RAM usage and speed up processes.

In [ ]:
# Turn on mixed precision training
# from tensorflow.keras import mixed_precision
# mixed precision is broken for EfficientNetB0 in TensorFlow, disabling
# mixed_precision.set_global_policy("mixed_float16") # set global data policy to mixed precision

## Build feature extraction model

In [ ]:
from tensorflow.keras import layers
# preprocessing not needed for EfficientNetB0
from tensorflow.keras.layers.experimental import preprocessing

# Create base model
input_shape = (224,224,3)
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False

# Create functional model
inputs = layers.Input(shape=input_shape, name="input_layer")
# Note: EfficientNetBX models have scaling layer built in, otherwise use below
# x = preprocessing.Rescaling(1./255) # 255 represents 0-255 for RGB values on pictures
x = base_model(inputs, training=False) # makes sure layers are in inference mode
x = layers.GlobalAveragePooling2D()(x)
# Dense layer as the final layer in broken
# x = layers.Dense(len(class_names)) # not using activation function here, we'll separate so we can conver to float32
outputs = layers.Dense(len(class_names), activation="softmax", name="softmax_float32")(x)
model = tf.keras.Model(inputs, outputs)

# compile the model
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])



## Setup Mixed Precision Training 

> Mixed Precision is currently broken for EfficientNetB0 with this version of TensorFlow

In [ ]:
model.summary()

## Checking layer dtype policies (checking if we're using mixed precision)

In [ ]:
# Check the dtype_policy attributes of the layers in the model
for layer in model.layers:
  print(layer.name, layer.trainable, layer.dtype, layer.dtype_policy)

# This is all float32 as mixed precision isn't currently functional for EfficientNetB0 in current TensorFlow version 2.9.2

In [ ]:
# Check the dtype_policy attributes of layers in the base model
for layer in model.layers[1].layers[:20]: # checks efficientnetB0 layers
  print(layer.name, layer.trainable, layer.dtype, layer.dtype_policy)

# again this is only for reference, mixed precision isn't currently working

## Fit the feature extraction model

Our goal is to fine-tune a pretrained model, we'll follow the order of:
1. Build a feature extraction model (train a couple of output layers with base layers frozen)
2. Fine-tune some of the base model layers

In [ ]:
# Fit feature extraction model with callbacks
history_101_food_classes_feature_extract = model.fit(train_data,
                                                     epochs=3,
                                                     steps_per_epoch=(len(train_data)),
                                                     validation_data=test_data,
                                                     validation_steps=(0.15 * len(test_data)),
                                                     callbacks=[create_tensorboard_callback(dir_name="training_logs",
                                                                                            experiment_name="efficientnetb0_101_classes_all_data_feature_extract"),
                                                                model_checkpoint])

In [ ]:
# Evaluate model on whole test dataset
results_feature_extract_model = model.evaluate(test_data)